# Importamos librerías - Creamos el dataframe de pericias

In [360]:
import pandas as pd
import numpy as np
from pprint import pprint

df = pd.read_csv('pericias_medicas.csv', sep=',',  encoding='utf-8')
#convierte en string los datos de la columna text
df = df['text'].apply(str)
df1 = df.head(20)
#len(df)

3806

## Limpieza de texto

In [4]:
# Limpieza de texto
# Primera etapa de la limpieza de texto
import re, string, unicodedata
'''
Se hace la eliminacion de texto entre corchetes, acentos, signos de puntuacion (excepto . y :), palabras con numeros.
Se eliminan los espacios de sobra
Se eliminan \r, \t, \v, \f, \a
'''
def limpiarTexto1(txt: str, bert=False, nums=False) -> str:
    """
    Elimina caracteres no deseados
    Params:
        **txt**:texto a ser limpiado de caracteres no desaeados
    """
    if nums:
        txt = re.sub(r'\d+', ' ', txt)
    if not bert:
        txt = txt.translate(str.maketrans(
            'áéíóúýàèìòùÁÉÍÓÚÀÈÌÒÙÝ', 'aeiouyaeiouAEIOUAEIOUY'))

    '''
    Eliminamos caracteres especiales: tabulador orizontal(\t), tabulador vertical(\v), 
    retorno de carro(\r), avance de pagina(\f), 
    caracter de retroceso: Marca el límite de una palabra(\b), 
    '''
    txt = txt.replace('\r', ' ').replace("\v", ' ').replace(
        "\t", ' ').replace("\f", ' ').replace("\a", ' ').replace("\b", ' ')
    txt = re.sub(' +', ' ', txt)
    txt = txt.strip()
    return txt
df1.apply(str)
df1 = df1.apply(limpiarTexto1)


In [5]:
# Segunda fase de limpieza
# Se eliminan todos los elementos que meten ruido al texto y que no fueron eliminados en la fase de limpieza 1.
import re

def limpiarTexto2(text):
    text = re.sub('^ ',' ',text)
    text = re.sub('\n +\n','\n',text)
    text = re.sub(' +\n\n','\n',text)
    text = re.sub('\n\n+','\n',text)
    text = re.sub(' \n','\n',text)
    text = re.sub('\d\n','',text)
    text = re.sub('\x0c','',text)
    text = re.sub('\u200b\n','',text)
    text = re.sub('\d{3,100}','',text)
    text = re.sub('\d+-\d+','',text)
    text = re.sub('\x0c','',text)
    text = re.sub('[nN]º|[nN][. ]º','',text)
    text = re.sub('[º<>/]','',text)
    text = re.sub('\d{3,100}','',text)
    text = re.sub('[a-zA-z-.]+@[a-zA-Z]+.com','',text)
    return text

dfLimpio = df1.apply(limpiarTexto2)

In [6]:
# Tercera fase de limpieza
# Eliminamos las lineas que no son de utilidad para el analisis o que van a afectar los resultados del mismo.
# Ejemplo de linea eliminada: las lineas que comienzan con "Se encuentra contestada en....."
import re

def limpiarTexto3(text):
    text = re.sub('[a-z1-9.]+[).-] [s|S]e encuentra contestad[a|o] .+[. \n]','',text)
    text = re.sub('[0-9]+[. ]+[yY]a fue contestado.+[.\n]','',text)
    text = re.sub('[fF]oja [1-9].+\n', '', text)
    text = re.sub('[pP]regunta[ 0-9]+[)].+\n|[rR]espuesta[ 0-9]+[)].+\n','',text)#elimina oraciones comenzadas en preguta/respuesta.
    return text

dfLimpio = dfLimpio.apply(limpiarTexto3)
dfLimpio = dfLimpio.apply(limpiarTexto2)

# Búsqueda de títulos

In [160]:
# Búsqueda de títulos
# Se crea un nuevo dataframe de títulos según distintos criterios

# busca titulos que comienzan con numeros romanos
def buscarTitulosRomanos(text):
    # Expresiones regulares para encontrar los numeros romanos
    tituloNroRomano =re.compile(r'(I{1,3}|IV|V|VI{1,3}|IX|X)[-.)][\w\s]+[:\n]')
    titulosRomanosEncontrados = []

    for m in tituloNroRomano.finditer(text):
        titulosRomanosEncontrados.append(m.group())

    return titulosRomanosEncontrados

# titulosConNroRomano: lista que guarda los titulos que comienzan con numeros romanos   
titulosConNroRomano=[]
for expediente in dfLimpio:
    titulosConNroRomano.append(buscarTitulosRomanos(expediente))


20

In [161]:
# Busca títulos que comienzan con números latinos
def buscarTitulosLatinos(text):
    # Expresiún regular para encontrar tútulos que comienzan con números
    tituloNroLatino =re.compile(r'[1-9][-.)][\w\s]+[:\n]')
    titulosLatinosEncontrados = []

    for m in tituloNroLatino.finditer(text):
        titulosLatinosEncontrados.append(m.group())

    return titulosLatinosEncontrados

# titulosConNroLatino: lista que guarda los tútulos que comienzan con números latinos
titulosConNroLatino=[]
for expediente in dfLimpio:
    titulosConNroLatino.append(buscarTitulosLatinos(expediente))

20

In [172]:
# Busca títulos en mayusculas
def buscarTitulosMayusculas(text):
    # Expresiín regular para encontrar títulos en mayusculas.
    tituloMayusculas =re.compile(r'([1-9]|I{1,3}|IV|V|VI{1,3}|IX|X)[-.)][A-Z -]+[:.\n]')
    titulosMayusculasEncontrados = []

    for m in tituloMayusculas.finditer(text):
        titulosMayusculasEncontrados.append(m.group())

    return titulosMayusculasEncontrados
# titulosMAyuscula: lista que guarda los títulos en mayusculas
titulosMayusculas=[]
for expediente in dfLimpio:
    titulosMayusculas.append(buscarTitulosMayusculas(expediente))

11

## Eliminación de stopwords. Nueva limpieza

In [ ]:
import nltk
import re

stops = nltk.corpus.stopwords.words('spanish')

# elimino stopwords
def remove_stops(texto: str) -> str:
    """
    Función que elimina stopwords
    Params:
        **texto**:texto a ser limpiado de stopwords

    """
    texto = [
        i for i in texto.split() if i not in stops
    ]
    return texto

def limpiarTitulos(text):
    text = text.lower()
    text = re.sub(r'\n','',text)
    text = re.sub(r'[a-z]\.[a-z]\.','',text)
    text = re.sub(r'(i{2,3}|iv|vi{1,3}|ix)','',text)
    text = re.sub(r'\W',' ',text)    
    text = re.sub(r'\d+','',text)
    text = re.sub(r' [a-z] ','',text)
    
    
    return text

# elimino stopwords y hago una última limpieza
titulosMayusculasStop = limpiarTitulos(str(titulosMayusculas))
titulosMayusculasStop = remove_stops(str(titulosMayusculasStop))
titulosConNroLatinoStop = limpiarTitulos(str(titulosConNroLatino))
titulosConNroLatinoStop = remove_stops(str(titulosConNroLatinoStop))
titulosConNroRomanoStop = limpiarTitulos(str(titulosConNroRomano))
titulosConNroRomanoStop = remove_stops(str(titulosConNroRomanoStop))
#pprint(titulosConNroRomanoStop)

# FASE DE ANALISIS

## Creación de dataframes de títulos

In [ ]:
# crear un df de títulos.
import pandas as pd

dfTitulos = pd.DataFrame()
dfTitulos['latinos'] = titulosConNroLatinoStop
dfTitulos['romanos'] = pd.DataFrame(titulosConNroRomanoStop)
dfTitulos['mayusculas'] = pd.DataFrame(titulosMayusculasStop)

## Contadores

In [359]:
# Contadores de títulos
contTitulosLatino = len(titulosConNroLatinoStop)
contTitulosRomanos = len(titulosConNroRomanoStop)
contTitulosmayusculas = len(titulosMayusculasStop)
totalTitulos = contTitulosLatino + contTitulosRomanos + contTitulosmayusculas

print(f'{contTitulosRomanos},   {contTitulosLatino},   {contTitulosmayusculas}')
print(f'TOTAL DE TITULOS ENCONTRADOS:   {totalTitulos}')

110,   581,   106
TOTAL DE TITULOS ENCONTRADOS:   797


### Dataframe de frecuencias

In [ ]:
# Obtengo la frecuencia relativa de las palabras de los titulos
frecuenciaTitulosRomanos = dfTitulos['romanos'].value_counts()
frecuenciaTitulosLatinos = dfTitulos['latinos'].value_counts()
frecuenciaTitulosMayuscula = dfTitulos['mayusculas'].value_counts()

# Crea un dataframe de frecuencias
dfFrecuencias = pd.DataFrame()
dfFrecuencias['latinos'] = frecuenciaTitulosLatinos
dfFrecuencias['romanos'] = pd.DataFrame(frecuenciaTitulosRomanos)
dfFrecuencias['mayuscula'] = pd.DataFrame(frecuenciaTitulosMayuscula)


## Creamos un vectorizador para ver las frecuencias de palabras por documento

In [361]:
# Importamos librerias necesarias.
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd
import nltk

stops = nltk.corpus.stopwords.words('spanish')

vectorizer = TfidfVectorizer(stop_words=set(stops))
X = vectorizer.fit_transform(df1)


In [362]:
true_k = 2
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)

order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names_out()

In [ ]:
for i in range(true_k): 
    print(f'Cluster %d:' % i, ), 
    for ind in order_centroids[i, :10]: 
        print(' %s' % terms[ind])   


print('\n') 
print('Predicción***************************') 
X = vectorizer.transform(['consideraciones medico legales']) 
predicted = model.predict(X)
print(predicted)